# Data Analysis Notebook (Group Analysis)

Comparison of P300 Amplitude Between Boundary and Non-Boundary trials: Focused on specific ERP components.

Paired T-tests, Graph the group ERPs: Statistical analysis and visualization of ERP data.

Topomap with sig electrodes: Spatial analysis of significant electrodes.

Correlation analyses: Exploring correlations between EEG and behavioral data.

In [6]:
# Import functions from previous notebook

### creating and ploting individual ERPs
%run '/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_postprocessing/2_Epoching_and_Behavioral_Integration.ipynb'

/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_52523/1323939628.py:25: RuntimeWarning: Mean of empty slice
  b_ERP = np.nanmean(b_epochs, axis=0)
/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_52523/1323939628.py:26: RuntimeWarning: Mean of empty slice
  nb_ERP = np.nanmean(nb_epochs, axis=0)


In [7]:
#List of subjects for group analysis
subject_list = ['sub-036', 'sub-037','sub-039', 'sub-040', 'sub-041', 'sub-042', 'sub-043',
                'sub-044', 'sub-045','sub-046', 'sub-047', 'sub-048', 'sub-049',
                'sub-050','sub-051', 'sub-052','sub-053', 'sub-054','sub-055','sub-056','sub-057',
                'sub-058','sub-061','sub-062','sub-064','sub-065','sub-066']

#Print out the current # of subjects
print(len(subject_list))


27


## Calculate P300 Amplitude for Boundary and Non-Boundary trials

In [9]:

#initiate the arraries dimension
nsubjects = len(subject_list)
nelectrodes=63
samples=1101

#Create empty arraries
all_b_ERP = np.zeros((nsubjects, nelectrodes, samples))
all_nb_ERP = np.zeros((nsubjects, nelectrodes, samples))

# do all the stuff above for all subjects in the list
for ctr_subj,isubj in enumerate(subject_list):
    
    raw, data_channels = load_and_process_eeg_data(data_folder, isubj)
    events, event_counts, event_id = read_event_markers(raw)
    epoch_data, condition_label, epochs = create_epochs(raw,events, event_id)
    merged_file, beh_path= load_beh(server_folder, isubj)
    b_ERP, nb_ERP, b_epochs, nb_epochs = create_indiv_epochs(epochs, events, event_id, merged_file, epoch_data, condition_label)
    #plot_indiv_erps(electrode_names, data_channels, b_ERP, nb_ERP, condition_label, data_folder, isubj)
    
    # Cube for ERPs for all subjects
    all_b_ERP[ctr_subj,:,:] = b_ERP #shape (nsubjects x nelectrodes x time in milliseconds)
    all_nb_ERP[ctr_subj,:,:] = nb_ERP

#save the results to local directory
all_b_ERP_path = '/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_postprocessing/group_ERPs/group_boundary.npy'
all_nb_ERP_path = '/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_postprocessing/group_ERPs/group_nonboundary.npy'

# Save the arrays
np.save(all_b_ERP_path, all_b_ERP)
np.save(all_nb_ERP_path, all_nb_ERP)

Opening raw data file /Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-036/eeg/postproc/sub-036_filtered_annot_crop.fif...


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_52523/3098042994.py:7: RuntimeWarning: This filename (/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/sub-036/eeg/postproc/sub-036_filtered_annot_crop.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fname)


    Range : 170000 ... 5330000 =    170.000 ...  5330.000 secs
Ready.
Used Annotations descriptions: ['Bit0/B  1', 'Bit1/B  1', 'Bit2/B  1', 'Bit3/B  1', 'Bit4/B  1', 'Bit5/B  1', 'Bit6/B  1']

Event counts: [480 480  15 105 105 270   1]
Not setting metadata
480 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 480 events and 1101 original time points ...
0 bad epochs dropped
/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/mergefile/mergefile_sub-036.csv
(480, 53)


ValueError: not enough values to unpack (expected 5, got 4)

In [21]:
######
# load the saved file from local directory
all_b_ERP = np.load('/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_postprocessing/group_ERPs/group_boundary.npy')
all_nb_ERP = np.load('/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_postprocessing/group_ERPs/group_nonboundary.npy')

#check the data shape (nsubjects x nelectrodes x time in milliseconds)
print(all_b_ERP.shape)
print(all_nb_ERP.shape)


(27, 63, 1101)
(27, 63, 1101)


In [25]:
# calculate the mean amplitudes across time window of interests
# selected time window: 300-700 ms after stimuli onsets

start_window = 400 
end_window = 800

# Calculate P300 for B, through averaging across time window
all_p3_b= np.nanmean(all_b_ERP[:,:,start_window:end_window],axis=2)
print(all_p3_b.shape) # shape: participants_num * electrode

# Calculate P300 for NB, through averaging across time window
all_p3_nb= np.nanmean(all_nb_ERP[:,:,start_window:end_window],axis=2)
print(all_p3_nb.shape)

(27, 63)
(27, 63)


/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_52523/2498727636.py:8: RuntimeWarning: Mean of empty slice
  all_p3_b= np.nanmean(all_b_ERP[:,:,start_window:end_window],axis=2)
/var/folders/np/qpdx19z16vj6zqjjk4pnx35r0000gn/T/ipykernel_52523/2498727636.py:10: RuntimeWarning: Mean of empty slice
  all_p3_nb= np.nanmean(all_nb_ERP[:,:,start_window:end_window],axis=2)


In [27]:
#set index for the electrode of interests
print(data_channels)
electrode_of_interest= 'Pz' 
electrode_index = data_channels.index(electrode_of_interest)  # Find the index of 'Cz'
print(electrode_index)

#Check the P3 for single electrode for all subjects
print('all_p3_b')
print(all_p3_b[:, electrode_index])

print('all_p3_nb')
print(all_p3_nb[:, electrode_index])

['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2']
12
all_p3_b
[10.43792507  2.71730729  1.39248893  3.90459399  2.92631755  4.68784444
  6.59245589 11.7783952   8.95618931  8.15954499  6.38001514  9.94618392
  8.30967157  2.90144275  8.60149844 10.94623946  2.8071328  11.30309929
  3.57846039  4.33774512  9.0986549   6.04750835 10.90133607  3.98360377
  6.1212748   4.63380503 19.3784489 ]
all_p3_nb
[ 2.24262613 -0.12483981 -0.38585823  0.99154748  3.33712179  2.20969232
 -1.63958477  1.69541686  1.7930716   2.66258277  2.0039419   1.10152877
  0.71178679  1.04673565  1.51234782  3.27332917 -0.22952249  0.21404261
  1.12129593  1

### Paired T-tests

In [29]:
# Run the Paired T-Test
(tstat,pval) = spst.ttest_rel(all_p3_b,
                              all_p3_nb,
                              axis=0, nan_policy='omit')

# Initialize an empty list to store electrode names with pval < 0.05
significant_electrodes = []

# Print the results, and create a list of significant_electrodes
for i, name in enumerate(data_channels):
    # Check if the p-value is less than 0.05
    if pval[i] < 0.05:
        print(f"Electrode: {name}, T-stat: {tstat[i]:.4f}, P-val: {pval[i]:.4f} *")
        significant_electrodes.append(True)
    else:
        print(f"Electrode: {name}, T-stat: {tstat[i]:.4f}, P-val: {pval[i]:.4f}")
        significant_electrodes.append(False)


Electrode: Fp1, T-stat: -0.8227, P-val: 0.4181
Electrode: Fz, T-stat: 2.2711, P-val: 0.0317 *
Electrode: F3, T-stat: -0.8707, P-val: 0.3933
Electrode: F7, T-stat: -0.2801, P-val: 0.7816
Electrode: FT9, T-stat: -0.0669, P-val: 0.9471
Electrode: FC5, T-stat: 0.9425, P-val: 0.3546
Electrode: FC1, T-stat: 3.9344, P-val: 0.0006 *
Electrode: C3, T-stat: 4.1135, P-val: 0.0003 *
Electrode: T7, T-stat: 0.9503, P-val: 0.3507
Electrode: TP9, T-stat: 0.2526, P-val: 0.8026
Electrode: CP5, T-stat: 3.9836, P-val: 0.0005 *
Electrode: CP1, T-stat: 6.9725, P-val: 0.0000 *
Electrode: Pz, T-stat: 7.8250, P-val: 0.0000 *
Electrode: P3, T-stat: 6.9820, P-val: 0.0000 *
Electrode: P7, T-stat: 4.9436, P-val: 0.0000 *
Electrode: O1, T-stat: 6.1345, P-val: 0.0000 *
Electrode: Oz, T-stat: 4.2429, P-val: 0.0002 *
Electrode: O2, T-stat: 4.4678, P-val: 0.0001 *
Electrode: P4, T-stat: 6.8828, P-val: 0.0000 *
Electrode: P8, T-stat: 2.8394, P-val: 0.0089 *
Electrode: TP10, T-stat: -1.1809, P-val: 0.2483
Electrode: CP6,

### Graph the grand ERPs

In [30]:
# calculate grand_ERPs (ERP average across all participants)
grand_ERP_nb = np.nanmean (group_nonboundary, axis=0) #(electrodes) * time points (63, 1101)
grand_ERP_b = np.nanmean (group_boundary, axis=0) 

#### Count the NA electrodes

In [31]:
# Assuming group_nonboundary has the shape (participants, electrodes, time points)
number_of_electrodes = group_nonboundary.shape[1]

# Initialize a 2D array to store the electrode index and the count of non-NaN participants
count_per_electrode = np.zeros((number_of_electrodes, 2), dtype=int)

# Iterate over all electrodes
for electrode in range(number_of_electrodes):
    # Count the number of non-NaN participants for this electrode
    count = (~np.isnan(group_nonboundary[:, electrode, :])).all(axis=1).sum()
    count_per_electrode[electrode] = [electrode, count]

# Now non_nan_count_per_electrode is a 2D array with the first column as electrode index
# and the second column as the count of non-NaN participants for that electrode
#print("Count of each electrode")
#print(count_per_electrode)

#### Calculate the error bars

In [32]:
import math

# shape should be (number_of_electrodes, number_of_time_points)
std_nb = np.nanstd(group_nonboundary, axis=0, ddof=1)  
std_b = np.nanstd(group_boundary, axis=0, ddof=1)  
# Assuming std is correctly calculated before this snippet
grand_ERP_nb_errorbar = np.zeros_like(std_nb) 
grand_ERP_b_errorbar = np.zeros_like(std_b) 

## nonboundary_errorbar
for electrode in range(len(count_per_electrode)):
    count = count_per_electrode[electrode, 1]  # Access the count for the electrode
    grand_ERP_nb_errorbar[electrode, :] = std_nb[electrode, :] / math.sqrt(count)
    #print(f"In nb condition, electrode {electrode} has {count} valid (non-NaN) observations.")


## boundary_errorbar
for electrode in range(len(count_per_electrode)):
    count = count_per_electrode[electrode, 1]  # Access the count for the electrode
    grand_ERP_b_errorbar[electrode, :] = std_b[electrode, :] / math.sqrt(count)
    #print(f"In b boundition, Electrode {electrode} has {count} valid (non-NaN) observations.")

#### Graph the Grand ERPs (with error bar)

In [ ]:
import seaborn as sns
import matplotlib.patches as mpatches

# Set Seaborn style
sns.set_style("whitegrid")

time_points = np.arange(grand_ERP_nb.shape[1])  # Assuming time points are linearly spaced

## grand_ERPs without error bars
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20, 10)) 

# Flatten the axs array for easy iteration
axs = axs.flatten()

for ax, electrode in zip(axs, electrode_names):
    electrode_idx = np.array(data_channels) == electrode
    # Extract the mean ERP and SEM for the current electrode
    mean_erp_b = grand_ERP_b[electrode_idx, :].flatten()
    sem_b = grand_ERP_b_errorbar[electrode_idx, :].flatten()
    
    mean_erp_nb = grand_ERP_nb[electrode_idx, :].flatten()
    sem_nb = grand_ERP_nb_errorbar[electrode_idx, :].flatten()
    
    # Plot mean ERP for boundary condition
    ax.plot(time_points, mean_erp_b, label='Boundary')
    # Add shaded SEM area for boundary condition
    ax.fill_between(time_points, mean_erp_b - sem_b, mean_erp_b + sem_b, alpha=0.2)
    
    # Plot mean ERP for nonboundary condition
    ax.plot(time_points, mean_erp_nb, label='NonBoundary')
    # Add shaded SEM area for nonboundary condition
    ax.fill_between(time_points, mean_erp_nb - sem_nb, mean_erp_nb + sem_nb, alpha=0.2)
    
    # Set the rest of the plot attributes
    ax.legend()
    ax.set_xlabel('Time Rel. Stimulus Onset (ms)')
    ax.set_ylabel('Voltage $\mu$V')
    ax.set_xticks([0, 100, 200, 400, 600, 800, 1000])  # Ensure these match your data
    ax.set_xticklabels([-100, 0, 100, 300, 500, 700, 900])  # Ensure these match your data
    ax.set_title(f'{condition_label}_{electrode}')
    ax.set_ylim(-10, 20)
    legend = ax.get_legend()
    if legend:
        legend.remove()
        
    # After plotting is done for all electrodes, create a legend for the error bars only
    boundary_patch = mpatches.Patch(color='blue', label='Boundary Error')
    nonboundary_patch = mpatches.Patch(color='orange',label='NonBoundary Error')
    # Add the custom legend to the current subplot
    ax.legend(handles=[boundary_patch, nonboundary_patch])
    
# Save and show the figure
figure_fname = os.path.join(data_folder, 'eeg_postprocessing', 'group_ERPs', 'Grand_ERPs_All_Electrodes_with_Error_Bars.pdf')
plt.savefig(figure_fname, bbox_inches='tight')

#### grand_ERPs without error bars

In [ ]:
## grand_ERPs without error bars
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20, 10)) 

# Flatten the axs array for easy iteration
axs = axs.flatten()

for ax, electrode in zip(axs, electrode_names):
    electrode_idx = np.array(data_channels) == electrode
    ax.plot(grand_ERP_b[electrode_idx,:].T)
    ax.plot(grand_ERP_nb[electrode_idx,:].T) # nb_ERP[electrode_idx,:].squeeze()
    ax.legend(['Boundary','NonBoundary'])
    ax.set_xlabel('Time Rel. Stimulus Onset (ms)'); 
    ax.set_ylabel('Voltage $\mu$V');
    ax.set_xticks([0,100,200,400,600,800,1000]); #time point of the vector
    ax.set_xticklabels([-100,0,100,300,500,700,900]); #actural time point
    ax.set_title(f'{condition_label}_{electrode}')
    ax.set_ylim(-10, 20)

# Remove unused subplots when cols are not filled
for i in range(len(electrode_names), len(axs)):
    fig.delaxes(axs[i])

# Adjust the space between the subplots
plt.subplots_adjust(wspace=0.3, hspace=0.3)

figure_fname = os.path.join(data_folder, 'eeg_postprocessing','group_ERPs' ,
                            f'Grand_ERPs_All_Electrodes.pdf')
# save the file 
plt.savefig(figure_fname, bbox_inches='tight')

## Topomap with sig electrodes

In [ ]:
# calculate grand_mean (P300 amplitude average across all participants)
grand_mean_diff = np.mean(grand_ERP_b [:,start_window:end_window],axis=1)- np.mean(grand_ERP_nb [:,start_window:end_window],axis=1)

# exclude broken electrode
grand_mean_diff[25]=0 #broken electrodes
grand_mean_diff[2]=0

#adjust the unit
grand_mean_diff=grand_mean_diff/1e6

#print out the difference
grand_mean_diff

In [ ]:
info = raw.info

# Create the EvokedArray, ensuring grand_mean_b is 2D (n_channels, 1)
grand_mean_2d = grand_mean_diff[:, np.newaxis]  # Add an axis to make it 2D

# Create the Evoked object
evoked_grand_mean = mne.EvokedArray(grand_mean_2d, info, tmin=0)

# Define a threshold and create the mask
#mask = evoked_grand_mean.data > 1e-13
significant_electrodes=np.array(significant_electrodes)
mask=significant_electrodes.reshape((63,1))

mask_params = dict(markersize=6, markerfacecolor="y")

# Plot the topomap for the grand mean across participants


im = evoked_grand_mean.plot_topomap(times=[0], ch_type='eeg', time_unit='s', colorbar=True,
                               mask=mask, mask_params=mask_params)

im.axes[0].set_title('300-700ms')
#plt.show()
figure_fname = os.path.join(data_folder, 'eeg_postprocessing', 'topo_maps',
                            f'Topo_maps_sig_electrodes.pdf')

# save the file 
plt.savefig(figure_fname, bbox_inches='tight')

# Close the figure window
plt.close()  

## Does the group_boundary - group_nonboundary negatively correlate with the b-nb temporal order? 

But maybe not at all the electrodes? Pz?

In [33]:
# Calculate the difference in mean amplitudes between conditions
difference_b_nb = all_p3_b - all_p3_nb # with a shape (subject, electrodes)
difference_b_nb.shape
 
#print the difference for specific electrode
print(difference_b_nb[:,18])

[ 5.77340218  2.72445218  2.1860605   3.01856793 -0.6991043   1.57177195
  6.96855339  4.21400397  4.41171296  4.33764029  1.1282172   7.56123146
  5.0658768  -0.02776577  6.69687263  7.02566108  2.67091255  9.39995692
  2.48298276  2.1133045   6.23012912  3.17128643  8.20249837  0.78999568
  8.40141255  2.93700568 15.32669281]


In [ ]:
# load beh_temporal_order file 
beh_order_accuracy = pd.read_csv('/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/group_results/order_accuracy.csv')
#print(beh_order_accuracy.shape) 

#calculate beh B-NB
beh_order_accuracy['B-NB'] = (beh_order_accuracy['B'] - beh_order_accuracy['NB'])
#beh_order_accuracy['B-NB'] = ((beh_order_accuracy['B'] - beh_order_accuracy['NB']))/beh_order_accuracy['NB']

#remove any subject in the beh data but not in the eeg subject list

beh_order_accuracy['Subject']= ["sub-0" + s.split()[1] for s in beh_order_accuracy['Subject']]
beh_order_accuracy = beh_order_accuracy[beh_order_accuracy['Subject'].isin(subject_list)]

#check if the leng of the lists matched
print("those two numbers should be the same")
print(len(subject_list))
print(len(beh_order_accuracy['Subject'])) # those two numbers should be the same
beh_order_accuracy.head(30)

In [ ]:
correlation_results = {}
from scipy.stats import pearsonr

# Iterate over each electrode
for i, electrode in enumerate(data_channels):
    # Extract the difference scores for this electrode across subjects
    diff_scores = difference_b_nb[:, i]
    
    # filter out the NaNs
    valid_indices = ~np.isnan(diff_scores) & ~np.isnan(beh_order_accuracy['B-NB'])
    diff_scores_filtered = diff_scores[valid_indices]
    beh_order_accuracy_filtered = beh_order_accuracy['B-NB'][valid_indices]
    
    #Run the correlation
    if len(diff_scores_filtered) > 1:  # Ensure there are at least two data points
        correlation, p_value = pearsonr(diff_scores_filtered, beh_order_accuracy_filtered)
        
        # Store the results
        correlation_results[electrode] = (correlation, p_value)
    else:
        correlation_results[electrode] = (np.nan, np.nan)  # Handle case with insufficient data

# print the correlation results
for electrode, (correlation, p_value) in correlation_results.items():
    print(f"Electrode: {electrode}, Correlation: {correlation:.4f}, P-value: {p_value:.4f}")

In [ ]:
import matplotlib.pyplot as plt

electrode_of_interest= ['Cz','Fz','Pz','P3','P4']

# Setup for plotting exactly 6 plots
num_plots = len(electrode_of_interest)
num_cols = 3  # Set the number of columns to 3 for 6 plots to be arranged in 2 rows
num_rows = 2  # Two rows to accommodate 6 plots

# Create a large figure to hold all subplots
plt.figure(figsize=(num_cols * 5, num_rows * 5))

# Iterate over each electrode of interest to create a subplot for each
for i, electrode in enumerate(electrode_of_interest):
    ax = plt.subplot(num_rows, num_cols, i + 1)
    
    # Find the index of the current electrode in the data_channels list
    electrode_index = data_channels.index(electrode)
    
    # Extract the difference scores for this electrode across subjects
    diff_scores = difference_b_nb[:, electrode_index]
    
    # Scatter plot of difference scores vs. B-NB accuracy scores
    ax.scatter(diff_scores, beh_order_accuracy['B-NB'], alpha=0.7)
    
    # Annotate the plot with the correlation coefficient and p-value
    # Ensure that correlation and p_value are correctly retrieved for each electrode
    correlation, p_value = correlation_results[electrode]
    ax.text(0.05, 0.95, f'r={correlation:.4f}, p={p_value:.4f}', transform=ax.transAxes, 
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Set plot titles and labels
    ax.set_title(f"Electrode: {electrode}")
    ax.set_xlabel('Difference B-NB (μV)')
    ax.set_ylabel('B-NB Score (%)')

# Ensure a clean layout
plt.tight_layout()


figure_fname = os.path.join(data_folder, 'eeg_postprocessing','correlation','order_correlation.pdf')
# save the file 
plt.savefig(figure_fname)

In [ ]:
diff_scores = difference_b_nb[:, 23]
print(diff_scores)

### Does the group_boundary - group_nonboundary negatively correlate with the b-nb source memory? 

But maybe not at all the electrodes? Pz?

In [ ]:
# load beh_temporal_order file 
beh_source_accuracy = pd.read_csv('/Volumes/ezzyatlab/experiments/NEvent/exp_eeg_v1/eeg_beh_results/group_results/source_accuracy.csv')
#print(beh_order_accuracy.shape) 

#calculate beh B-NB
beh_source_accuracy['B-NB'] = (beh_source_accuracy['B'] - beh_source_accuracy['NB'])
#
#beh_order_accuracy['B-NB'] = ((beh_order_accuracy['B'] - beh_order_accuracy['NB']))/beh_order_accuracy['NB']

#remove any subject in the beh data but not in the eeg subject list

beh_source_accuracy['Subject']= ["sub-0" + s.split()[1] for s in beh_source_accuracy['Subject']]

#remove any subject in the beh data but not in the eeg subject list
beh_source_accuracy = beh_source_accuracy[beh_source_accuracy['Subject'].isin(subject_list)]
print(len(subject_list))
print(len(beh_source_accuracy['Subject'])) # those two numbers should be the same
beh_source_accuracy.head(5)

In [ ]:
correlation_results = {}
from scipy.stats import pearsonr

# Iterate over each electrode
for i, electrode in enumerate(data_channels):
    # Extract the difference scores for this electrode across subjects
    diff_amplitude = difference_b_nb[:, i]
    
    # filter out the NaNs
    valid_indices = ~np.isnan(diff_scores) & ~np.isnan(beh_source_accuracy['B-NB'])
    diff_scores_filtered = diff_scores[valid_indices]
    beh_source_accuracy_filtered = beh_source_accuracy['B-NB'][valid_indices]
    
    #Run the correlation
    if len(diff_scores_filtered) > 1:  # Ensure there are at least two data points
        correlation, p_value = pearsonr(diff_scores_filtered, beh_source_accuracy_filtered)
        
        # Store the results
        correlation_results[electrode] = (correlation, p_value)
    else:
        correlation_results[electrode] = (np.nan, np.nan)  # Handle case with insufficient data


# print the correlation results
for electrode, (correlation, p_value) in correlation_results.items():
    print(f"Electrode: {electrode}, Correlation: {correlation:.4f}, P-value: {p_value:.4f}")
    
---
correlation_results = {}
from scipy.stats import pearsonr

# Iterate over each electrode
for i, electrode in enumerate(data_channels):
    # Extract the difference scores for this electrode across subjects
    diff_scores = difference_b_nb[:, i]
    
    # filter out the NaNs
    valid_indices = ~np.isnan(diff_scores) & ~np.isnan(beh_order_accuracy['B-NB'])
    diff_scores_filtered = diff_scores[valid_indices]
    beh_order_accuracy_filtered = beh_order_accuracy['B-NB'][valid_indices]
    
    #Run the correlation
    if len(diff_scores_filtered) > 1:  # Ensure there are at least two data points
        correlation, p_value = pearsonr(diff_scores_filtered, beh_order_accuracy_filtered)
        
        # Store the results
        correlation_results[electrode] = (correlation, p_value)
    else:
        correlation_results[electrode] = (np.nan, np.nan)  # Handle case with insufficient data

# print the correlation results
for electrode, (correlation, p_value) in correlation_results.items():
    print(f"Electrode: {electrode}, Correlation: {correlation:.4f}, P-value: {p_value:.4f}")

In [ ]:
electrode_of_interest= ['Cz','Fz','Pz','P3','P4']

# Setup for plotting exactly 6 plots
num_plots = len(electrode_of_interest)
num_cols = 3  # Set the number of columns to 3 for 6 plots to be arranged in 2 rows
num_rows = 2  # Two rows to accommodate 6 plots

# Create a large figure to hold all subplots
plt.figure(figsize=(num_cols * 5, num_rows * 5))

# Iterate over each electrode of interest to create a subplot for each
for i, electrode in enumerate(electrode_of_interest):
    ax = plt.subplot(num_rows, num_cols, i + 1)
    
    # Find the index of the current electrode in the data_channels list
    electrode_index = data_channels.index(electrode)
    
    # Extract the difference scores for this electrode across subjects
    diff_scores = difference_b_nb[:, electrode_index]
    
    # Scatter plot of difference scores vs. B-NB accuracy scores
    ax.scatter(diff_scores, beh_source_accuracy['B-NB'], alpha=0.7)
    
    # Annotate the plot with the correlation coefficient and p-value
    # Ensure that correlation and p_value are correctly retrieved for each electrode
    correlation, p_value = correlation_results[electrode]
    ax.text(0.05, 0.95, f'r={correlation:.4f}, p={p_value:.4f}', transform=ax.transAxes, 
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Set plot titles and labels
    ax.set_title(f"Electrode: {electrode}")
    ax.set_xlabel('Difference B-NB (μV)')
    ax.set_ylabel('B-NB Source accuracy (%)')

# Ensure a clean layout
plt.tight_layout()


figure_fname = os.path.join(data_folder, 'correlation_Source.pdf')
# save the file 
plt.savefig(figure_fname)